In [1]:
import os

import numpy as np
import torch

from src.train import train_with_embeddings
from src.utils import cross_entropy

%load_ext autoreload
%autoreload 2

# Training

In [2]:
data_path = 'data/train/nsfw_trainset'
x_drawings = np.load(os.path.join(data_path, 'drawings', 'img_emb', 'img_emb_0.npy'))
x_hentai = np.load(os.path.join(data_path, 'hentai', 'img_emb', 'img_emb_0.npy'))
x_netural = np.load(os.path.join(data_path, 'neutral', 'img_emb', 'img_emb_0.npy'))
x_porn = np.load(os.path.join(data_path, 'porn', 'img_emb', 'img_emb_0.npy'))
x_sexy = np.load(os.path.join(data_path, 'sexy', 'img_emb', 'img_emb_0.npy'))

In [3]:
len(x_drawings), len(x_hentai), len(x_netural), len(x_porn), len(x_sexy)

(41370, 30367, 64491, 70681, 35158)

In [4]:
y = np.concatenate(
        (np.zeros((len(x_drawings) + len(x_netural), 1)), 
         np.ones((len(x_hentai) + len(x_porn) + len(x_sexy), 1))), axis=0)
X = np.concatenate((x_drawings, x_netural, x_hentai, x_porn, x_sexy), axis=0)

In [5]:
model = train_with_embeddings(X, y, 'ViT-L-14')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 757/757 [00:01<00:00, 454.87it/s]


Epoch 1 | Train Loss 0.327259197476833 | Val Loss 0.08895397911730565


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 757/757 [00:01<00:00, 517.51it/s]


Epoch 2 | Train Loss 0.08433198034418858 | Val Loss 0.08446082378688612


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 757/757 [00:01<00:00, 498.21it/s]


Epoch 3 | Train Loss 0.08179516677684402 | Val Loss 0.08403973024534551


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 757/757 [00:01<00:00, 497.35it/s]


Epoch 4 | Train Loss 0.0815100309708435 | Val Loss 0.0839939782297925


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 757/757 [00:01<00:00, 499.08it/s]


Epoch 5 | Train Loss 0.08148453900863661 | Val Loss 0.08399243546943916


# Testing 

In [6]:
data_path = 'data/test/nsfw_testset'
x_drawings = np.load(os.path.join(data_path, 'drawings-test', 'img_emb', 'img_emb_0.npy'))
x_hentai = np.load(os.path.join(data_path, 'hentai-test', 'img_emb', 'img_emb_0.npy'))
x_netural = np.load(os.path.join(data_path, 'neutral-test', 'img_emb', 'img_emb_0.npy'))
x_porn = np.load(os.path.join(data_path, 'porn-test', 'img_emb', 'img_emb_0.npy'))
x_sexy = np.load(os.path.join(data_path, 'sexy-test', 'img_emb', 'img_emb_0.npy'))

In [7]:
x_drawings.shape[0], x_hentai.shape[0], x_netural.shape[0], x_porn.shape[0], x_sexy.shape[0]

(853, 126, 766, 760, 694)

In [8]:
y = np.concatenate(
        (np.zeros((len(x_drawings) + len(x_netural) , 1)), 
         np.ones((len(x_hentai) + len(x_porn) + len(x_sexy), 1))), axis=0)
X = np.concatenate((x_drawings, x_netural, x_hentai, x_porn, x_sexy), axis=0)

In [9]:
X = torch.tensor(X, dtype=torch.float)
X = torch.nn.functional.normalize(X)
y = torch.tensor(y, dtype=torch.float).cuda()

## Load and Test Trained MLP Model

In [10]:
model.eval()
with torch.no_grad():
    y_p = model(X.cuda())
cross_entropy(y, y_p)

tensor(0.1328, device='cuda:0')

## Load and Test Provided MLP Model

In [11]:
model.load_state_dict(torch.load('./models/clip_ViT-L-14_openai_binary_nsfw_head.pth'))

model.eval()
with torch.no_grad():
    y_p = model(X.cuda())
cross_entropy(y, y_p)

tensor(0.1455, device='cuda:0')